# CIS twinning - Approximate

In [1]:
from interfacemaster.cellcalc import get_primitive_hkl
from interfacemaster.interface_generator import core, print_near_axis, convert_vector_index
from numpy import array, dot, round, var, average
from numpy.linalg import inv

## Define an interface core class

Input two cif files.For GBs, input the same file; for interfaces, input two different files.


The first structure is non-changed while the second one can be deformed to form an approximate CSL

Please input files of CONVENTIONAL cells if you are not familiar with the indices in the primitive cell.

In [2]:
my_interface = core('cif_files/CIS-exp.cif','cif_files/CIS-exp.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


## Specify the rotation axis and searching parameters to find an approximate CSL

Rotation axis should be expressed in the primitive cell frame, we can convert this.

For example we need an axis [2,-2,1] expressed in the conventional cell.

In [3]:
axis = convert_vector_index(my_interface.conv_lattice_1, my_interface.lattice_1, [2,-2,1])
axis

array([ 3., -1.,  2.])

For the meaning of searching parameters, please check the paper.

If you are making an exact CSL, just set du, S & dd to be 'very small'. 

In this case we make a known twinning GB, which is an exact CSL.

In [4]:
my_interface.parse_limit(du = 1e-2, S  = 1e-2, sgm1=100, sgm2=100, dd = 1e-2)

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
my_interface.search_one_position(axis,180,1,0.01)

[ 11.562  -11.562   11.6422]
Congrates, we found an appx CSL!

U1 = 
[[ 0  1  1]
 [-1  1 -1]
 [-1  0  1]]; sigma_1 = 3

U2 = 
[[ 0 -1  2]
 [ 1 -1  0]
 [ 1  0  1]]; sigma_2 = 3

D = 
[[ 1.0030793  -0.0030793  -0.00611617]
 [-0.0030793   1.0030793   0.00611617]
 [ 0.00310066 -0.00310066  0.9938414 ]]

axis = [ 11.562  -11.562   11.6422] ; theta = 180.0



'U1', 'U2' are two sets of indices of the CSL in the two crystals' frames,
'D' is the deformation applied to the crystal 2 to form an approximate CSL

As you can see, to form a twinning structure we need to deform the structure provided!

## Convert the miller indices to be expressed in the primitive cell

Now we need to specify the orientation of our interface, which is by giving a miller indices in expressed in the primitive cell.

In [5]:
"""
The interface plane is the [1,-1,2] plane in the conventional cell and now we \
transform it into primitive cell
"""
hkl = get_primitive_hkl(array([1,-1,2]), my_interface.conv_lattice_1, my_interface.lattice_1)
hkl

array([ 1, -1,  1])

## Compute the indices of the two slabs to make bicrystal

Now let's compute the indices of the two slabs forming the interface
If you want a near orthogonal bicrystal, please set 'orthogonal to yes, 
and adjust the lim & tol.

In [6]:
my_interface.compute_bicrystal(hkl, normal_ortho = True, lim = 50, tol_ortho = 1e-2)

cell 1:
[[ 3  0  1]
 [-1 -1  1]
 [ 2 -1  0]]
cell 2:
[[ 3  0 -1]
 [-1  1 -1]
 [ 2  1  0]]


Here we obtained the two sets of indices to build a bicrystal

## Make a bicrystal!

You can adjust the bicrystal by
1. do expansion by set 'xyz' = [dimX, dimY, dimZ]
2. do RBT by 'dzdy' = a translation vector 
(please confine dzdy in the interface plane, 
for RBT in the normal, please use 'dp1' and 'dp2', to make vacuum in the interface use 'dx');
3. introduce vacuum in the interface by 'dx' = the distance spacing the two crystals
4. adjust the termination by 'dp1', 'dp2' = length of shift of the position of 
the termination from the interface into the bulk
5. introduce vacuum to make a surface in the end of the bicrystal by
'vx' = length of the vaccumn

default: xyz = [1,1,1] and others = 0

In [7]:
my_interface.get_bicrystal(xyz_1 = [1,1,1], xyz_2 = [1,1,1])

Good! You have made a 'POSCAR' file of this bicrystal, we also generated two files named 'cell_1.cif' and 'cell_2.cif' providing the structure of the two slabs.

In [8]:
#To sample the CNID, try
my_interface.sample_CNID(grid = [20,20], xyz_1 = [1,1,1], xyz_2 = [1,1,1])

CNID
[[ 0. -1.]
 [-1. -1.]
 [-1.  0.]]
making 400 files...
completed


Please check the generated 'POSCAR.x.y' files. 
The CNID here is expressed in the primitive cell's frame.
You can refer to the cartesian coordinates or in the conventional cell's frame

In [9]:
CNID_cartesian = round(my_interface.CNID,8)
CNID_cartesian

array([[ 2.8905, -5.781 ],
       [-2.8905, -5.781 ],
       [-5.8211, -0.    ]])

In [10]:
CNID_conv = round(dot(inv(my_interface.conv_lattice_1)\
                              , my_interface.CNID),8)
CNID_conv

array([[ 0.5, -1. ],
       [-0.5, -1. ],
       [-0.5,  0. ]])